In [1]:
import os
import re

In [2]:
import lib.cnn.matnpyio as io
import numpy as np
import pandas as pd

In [3]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
# path to the csv files 
result_path =  '/home/rudy/Python2/regression_linear/result_singe2/' 

# where to save the fig and numpy files
figure_path = '/home/rudy/Python2/regression_linear/' + '/figure_singe2_ridge/'

# path to raw data (in order to get areas list)
#base_path = '/media/rudy/disk2/lucy/'
base_path = '/media/rudy/disk2/ethyl/'

In [ ]:
# dict[area] == cortex
dico_area_to_cortex = io.get_dico_area_to_cortex()

In [ ]:
file_names = os.listdir(result_path)

In [ ]:
file_names

In [ ]:
# get list of session 
# for ridge
session = []
for txt in file_names :
    if txt[-9:] == 'ridge.csv':
        session.append(re.findall('\d+', txt)[0])

In [4]:
###

In [ ]:

for sess_no in session :
    print(sess_no)
    
    #################
    # LOAD CSV FILE #
    #################
    
    # recording info path
    rinfo_path = base_path +sess_no+'/session01/' + 'recording_info.mat'
    # file path
    file_name = result_path+ 'result_sess_no_'+sess_no+'channel_to_channel_all_interval_with_error_bar_ridge.csv'
       
    # load csv file 
    df = pd.read_csv(file_name)
    
    # select data
    only_correct_trials = True

    lowcut1, highcut1, order1 = 7, 12, 3
    lowcut2, highcut2, order2 = 7, 12, 3

    window_size1 = 200
    window_size2 = window_size1 #window_size1 ### 0.72
    step = 100
    delay = 0

    str_freq1 = 'low'+str(lowcut1)+'high'+str(highcut1)+'order'+str(order1)
    str_freq2 = 'low'+str(lowcut2)+'high'+str(highcut2)+'order'+str(order2)
    
    df_cortex = df[(df['str_freq1'] == str_freq1) & (df['str_freq2'] == str_freq2) & (df['window_size1'] == window_size1) & (df['window_size2']==window_size2)]
    
    # select channels
    
    target_cortex1 = ['Visual', 'Prefrontal', 'Parietal', 'Motor', 'Somatosensory']
    target_cortex2 = ['Visual', 'Prefrontal', 'Parietal', 'Motor', 'Somatosensory']

    target_areas1 = []
    for cortex1 in target_cortex1 :
        for area in io.get_area_cortex(rinfo_path, cortex1, unique = False):
            target_areas1.append(area)
    target_areas2 = []
    for cortex2 in target_cortex2 :
        for area in io.get_area_cortex(rinfo_path, cortex2, unique = False):
            target_areas2.append(area)

    target1 = collections.Counter(target_areas1)
    target2 = collections.Counter(target_areas2)

    label1 = []
    for area1 in target1 :
        for count1 in range(target1[area1]):
            label1.append(area1)

    label2 = []
    for area2 in target2 :
        for count2 in range(target2[area2]):
            label2.append(area2)
            
    # easier to select idx with numpy array            
    label1 = np.array(label1)
    label2 = np.array(label2)
    

    # build matrix with all channels
    FC = np.zeros( (len(label1), len(label2)))

    i=0
    for area1 in target1 :
        for count1 in range(target1[area1]):
            j=0
            for area2 in target2 :
                for count2 in range(target2[area2]):

                    r = df_cortex[ (df_cortex['area1'] == area1) & (df_cortex['num1'] == count1) & (df_cortex['area2'] == area2) & (df_cortex['num2'] == count2)]['r2_test'].values
                    if len(r) > 0 :
                        #print(i,j)
                        FC[i,j] = r[0]

                    else :
                        print(area1, area2)


                    j+=1
            i+=1
            
    # show matrix 
    fig, ax = plt.subplots(figsize=(15,15))
    im = ax.imshow(FC, vmin=0)#, vmax=1.0)# cmap='jet')
    fig.colorbar(im)


    # We want to show all ticks...
    ax.set_xticks(np.arange(len(label2)))
    ax.set_yticks(np.arange(len(label1)))
    # ... and label them with the respective list entries
    ax.set_xticklabels(label2, linespacing=5)
    ax.set_yticklabels(label1, linespacing=5)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=0, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    # for i in range(len(label1)):
    #     for j in range(len(label2)):
    #         if round(FC[i, j],3) < 0.8* FC.max() :
    #             text = ax.text(j, i, round(FC[i, j],1),
    #                            ha="center", va="center", color="w")
    #         else:
    #             text = ax.text(j, i, round(FC[i, j],1),
    #                            ha="center", va="center", color="b")


    ax.set_title("R_squared")
    #fig.tight_layout()
    
    # save matrix and labels as numpy file (make directory if not exists)
    
    # create directory if not exists
    directory = figure_path +sess_no+'/'
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    # save as numpy files
    np.save(directory +'channel_to_channel_all_cortex_low_'+str(lowcut1)+'_high_'+str(highcut1)+'.npy', FC )
    np.save(directory +'label1.npy', label1 )
    np.save(directory +'label2.npy', label2 )

    # save plot as jpg file
    plt.savefig(directory+'channel_to_channel_all_channel_low_'+str(lowcut1)+'_high_'+str(highcut1)+'.jpg')
    plt.show()
    
    
    ###########################
    #   SAVE TIME COURSE      #
    ###########################
    
    # get number of step
    step_list = eval( df['step_list'].values[0] )
    # number of time step (always the same)
    n_time = len(step_list)
    
    # Create matrix and fill them
    FC_time_course = np.zeros( (len(label1), len(label2), n_time))
    FC_time_course_error_bar = np.zeros( (len(label1), len(label2), n_time))
    
    for area1 in target1 :
        for count1 in range(target1[area1]):
            j=0
            for area2 in target2 :
                for count2 in range(target2[area2]):
                    r2_test_time = eval(df['r2_test_time'].values[0])

                    r2_test_time = df_cortex[ (df_cortex['area1'] == area1) & (df_cortex['num1'] == count1) & (df_cortex['area2'] == area2) & (df_cortex['num2'] == count2)]['r2_test_time'].values
                    r2_error_bar_test_time = df_cortex[ (df_cortex['area1'] == area1) & (df_cortex['num1'] == count1) & (df_cortex['area2'] == area2) & (df_cortex['num2'] == count2)]['r2_error_bar_test_time'].values
                    if len(r2_test_time) > 0 :
                        #print(i,j)
                        r2_test_time = eval(r2_test_time[0])
                        r2_error_bar_test_time = eval(r2_error_bar_test_time[0])

                        for t in range(len(r2_test_time)):
                            FC_time_course[i,j,t] = r2_test_time[t]
                            FC_time_course_error_bar[i,j,t] = r2_error_bar_test_time[t] 

                    else :
                        print(area1, area2)
                        
#     directory = figure_path +sess_no+'/'
#     if not os.path.exists(directory):
#         os.makedirs(directory)
        
    np.save(directory +'channel_to_channel_all_cortex_low_'+str(lowcut1)+'_high_'+str(highcut1)+'_time_course.npy', FC_time_course )
    np.save(directory +'channel_to_channel_all_cortex_low_'+str(lowcut1)+'_high_'+str(highcut1)+'_time_course_error_bar.npy', FC_time_course_error_bar )
#     np.save(directory +'label1.npy', label1 )
#     np.save(directory +'label2.npy', label2 )

    
    
    
#     ###########################
#     # SAVE CORTEX BY CORTEX   #
#     ###########################
    
    
#     # get cortex of each channel
#     label1_cortex = []
#     for area in label1 :
#         label1_cortex.append(dico_area_to_cortex[area])

#     label2_cortex = []
#     for area in label2 :
#         label2_cortex.append(dico_area_to_cortex[area])

#     label1_cortex = np.array(label1_cortex)
#     label2_cortex = np.array(label2_cortex)
# #     label1 = np.array(label1)
# #     label2 = np.array(label2)
    
#     # plot matrix cortex by cortex + save it
#     # only plot r2 score if r2 > threshold : there is a link between the input and the output
#     threshold = df_cortex['r2_error_bar_test'].max()
#     for cortex1 in ['Visual', 'Parietal', 'Motor', 'Somatosensory']:  #['Visual', 'Prefontal', 'Parietal', 'Motor', 'Somatosensory']: 
#         ind1 = (label1_cortex ==cortex1)
#         for cortex2 in ['Visual', 'Parietal', 'Motor', 'Somatosensory']:  #['Visual', 'Prefontal', 'Parietal', 'Motor', 'Somatosensory'] :
#             print(cortex1, ' to ', cortex2)
#             ind2 = (label2_cortex==cortex2)
            
#             # select cortex
#             label1_bis = label1[ind1]
#             label2_bis = label2[ind2]
#             if len(label1_bis) ==0 or len(label2_bis)==0:
#                 continue

#             FC_cortex1_to_cortex2 = FC[ind1, :][:,ind2]
            
#             # save matrix and make directory if not exists            
#             if not os.path.exists(directory+cortex1+'_to_'+cortex2+'/'):
#                 os.makedirs(directory+cortex1+'_to_'+cortex2+'/')
#             np.save(directory + cortex1+'_to_'+cortex2+'/channel_to_channel_'+cortex1+'_to_'+cortex2+'__low_'+str(lowcut1)+'_high_'+str(highcut1)+'.npy', FC_cortex1_to_cortex2 )
#             np.save(directory + cortex1+'_to_'+cortex2+'/label1.npy', label1_bis)
#             np.save(directory + cortex1+'_to_'+cortex2+'/label2.npy', label2_bis)
            


#             fig, ax = plt.subplots(figsize=(15,15))
#             #fig, ax = plt.subplots()
#             im = ax.imshow(FC_cortex1_to_cortex2, vmin=0)#, vmax=1.0)# cmap='jet')
#             # make the size of the colorbar the same as the picture 
#             divider = make_axes_locatable(ax)
#             cax = divider.append_axes("right", size="5%", pad=0.10)
#             fig.colorbar(im, cax=cax)


#             # We want to show all ticks...
#             ax.set_xticks(np.arange(len(label2_bis)))
#             ax.set_yticks(np.arange(len(label1_bis)))
#             # ... and label them with the respective list entries
#             ax.set_xticklabels(label2_bis)
#             ax.set_yticklabels(label1_bis)

#             # Rotate the tick labels and set their alignment.
#             plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
#                      rotation_mode="anchor")

#             #Loop over data dimensions and create text annotations.
#             for i in range(len(label1_bis)):
#                 for j in range(len(label2_bis)):
#                      if FC_cortex1_to_cortex2[i, j] > threshold :
#                         # plot round number 
#                         # and with black or white according to the color of the pixel

#                         if round(FC_cortex1_to_cortex2[i, j],3) < 0.8* FC_cortex1_to_cortex2.max() :

#                             text = ax.text(j, i, round(FC_cortex1_to_cortex2[i, j],1),
#                                            ha="center", va="center", color="w")
#                         else:
#                             text = ax.text(j, i, round(FC_cortex1_to_cortex2[i, j],1),
#                                            ha="center", va="center", color="b")


#             ax.set_title("R_squared")
#             #fig.tight_layout()
            
#             # save figure
#             #plt.savefig(directory + cortex1+'_to_'+cortex2+'/channel_to_channel_'+ cortex1+'_to_'+cortex2+'_low_'+str(lowcut1)+'_high_'+str(highcut1)+'.jpg' )
#             plt.show()    

In [ ]:
    step_list = eval( df['step_list'].values[0] )
    # number of time step (always the same)
    n_time = len(step_list)
    # correct a mistake => match onset = 2000 (and sample onset = 0)
    for i in range(18, len(step_list)):
        step_list[i] = step_list[i] + 200


    # select channels

    target_cortex1 = ['Visual', 'Prefontal', 'Parietal', 'Motor', 'Somatosensory']
    target_cortex2 = ['Visual', 'Prefontal', 'Parietal', 'Motor', 'Somatosensory']

    target_areas1 = []
    for cortex1 in target_cortex1 :
        for area in io.get_area_cortex(rinfo_path, cortex1, unique = False):
            target_areas1.append(area)
    target_areas2 = []
    for cortex2 in target_cortex2 :
        for area in io.get_area_cortex(rinfo_path, cortex2, unique = False):
            target_areas2.append(area)

    target1 = collections.Counter(target_areas1)
    target2 = collections.Counter(target_areas2)

    label1 = []
    for area1 in target1 :
        for count1 in range(target1[area1]):
            label1.append(area1)

    label2 = []
    for area2 in target2 :
        for count2 in range(target2[area2]):
            label2.append(area2)

    # easier to select idx with numpy array            
    label1 = np.array(label1)
    label2 = np.array(label2)

    # print something 

    #     print(label1)
    #     print(label2)

    #     print(len(label1))
    #     print(len(label2))

    # build matrix with all channels
    
        step_list = eval( df['step_list'].values[0] )
    # number of time step (always the same)
    n_time = len(step_list)
    FC = np.zeros( (len(label1), len(label2), n_time))
    FC_error_bar = np.zeros( (len(label1), len(label2), n_time))

    i=0
    for area1 in target1 :
        for count1 in range(target1[area1]):
            j=0
            for area2 in target2 :
                for count2 in range(target2[area2]):
                    r2_test_time = eval(df['r2_test_time'].values[0])

                    r2_test_time = df_cortex[ (df_cortex['area1'] == area1) & (df_cortex['num1'] == count1) & (df_cortex['area2'] == area2) & (df_cortex['num2'] == count2)]['r2_test_time'].values
                    r2_error_bar_test_time = df_cortex[ (df_cortex['area1'] == area1) & (df_cortex['num1'] == count1) & (df_cortex['area2'] == area2) & (df_cortex['num2'] == count2)]['r2_error_bar_test_time'].values
                    if len(r2_test_time) > 0 :
                        #print(i,j)
                        r2_test_time = eval(r2_test_time[0])
                        r2_error_bar_test_time = eval(r2_error_bar_test_time[0])

                        for t in range(len(r2_test_time)):
                            FC[i,j,t] = r2_test_time[t]
                            FC_error_bar[i,j,t] = r2_error_bar_test_time[t] 

                    else :
                        print(area1, area2)


                    j+=1
            i+=1
            
    step_list = np.array(step_list)
    
    directory = result_path + 'figure_singe2_ridge/'+sess_no+'/'    
    np.save(directory +'channel_to_channel_all_cortex_low_'+str(lowcut1)+'_high_'+str(highcut1)+'time_course.npy', FC )
    np.save(directory +'channel_to_channel_all_cortex_low_'+str(lowcut1)+'_high_'+str(highcut1)+'time_course_error_bar.npy', FC_error_bar )
    np.save(directory +'time_for_time_course.npy', step_list )